In [3]:
import ckanapi
from ckanapi.errors import NotFound,CKANAPIError, NotAuthorized
import pandas as pd
import requests
from requests_cache  import CachedSession

In [4]:
session = CachedSession(
    expire_after = 600
)
try:
    ua_portal = ckanapi.RemoteCKAN('https://data.carpathia.gov.ua/')
    datasetTwo = ua_portal.action.package_show(id='f9a3dd3a-0204-490d-b6e0-013ddecfcc4c')
    print(f'Title: {datasetTwo['title']}')

    resources_url = None
    for rec in datasetTwo['resources']:
        
        if rec['format'].lower() == 'geojson':
            resources_url = rec['url']
            break

    if resources_url:
        response = session.get(resources_url)
        response.raise_for_status()

        geo_data =  response.json()  
        df_bombshelter = pd.json_normalize(geo_data['features'])         
except NotFound:
    print("Error: ID of dataset doesn`t exist on this site;")
except NotAuthorized:
    print("Error: This dataset is private or requires API key to access;")
except CKANAPIError as e:
    print(f"Error:A CKAN API error occured: {e}; ")
except Exception as e:
    print(f"An unexpected error ocurred: {e};")

Title: Адміністративні дані, що збираються (обробляються) та підлягають оприлюдненню управлінням цивільного захисту


In [18]:
print(df_bombshelter.columns)
print(df_bombshelter.head())

Index(['type', 'properties.OTG', 'properties.Number', 'properties.City',
       'properties.Name', 'properties.Area', 'properties.People',
       'properties.TypeZs', 'properties.Property', 'properties.Adress',
       'properties.Rajon', 'properties.Type', 'properties.Bezbar',
       'geometry.type', 'geometry.coordinates'],
      dtype='object')
      type properties.OTG  properties.Number properties.City  \
0  Feature    Мукачівська                  0     м.Мукачево,   
1  Feature      Тячівська                  0         м.Тячів   
2  Feature    Мукачівська                  0     м. Мукачево   
3  Feature    Ужгородська              24065       м.Ужгород   
4  Feature    Мукачівська              24416     м. Мукачево   

                   properties.Name properties.Area  properties.People  \
0                        “Зі-Схід”              60                100   
1                   Ліцей-інтернат             395                150   
2                      “Зі-Північ”             

In [ ]:

df_categorized = df_bombshelter[(df_bombshelter["properties.City"] == "м. Мукачево")]
display(df_categorized)


,type,properties.OTG,properties.Number,properties.City,properties.Name,properties.Area,properties.People,properties.TypeZs,properties.Property,properties.Adress,properties.Rajon,properties.Type,properties.Bezbar,geometry.type,geometry.coordinates
282,Feature,Мукачівська,0,м. Мукачево,СШ №16,285,473,вбудоване,Комунальна,"вул.Шевченка,68",Мукачівський,Найпростіші укриття,true,Point,"[22.702505, 48.434821]"
484,Feature,Мукачівська,0,м. Мукачево,Мукачівська спеціалізована школа І-ІІІ ст.,138,230,вбудоване,Державна,"вул.Болгарська,2",Мукачівський,Найпростіші укриття,true,Point,"[22.694828, 48.44222]"


In [17]:
df_cityName =df_bombshelter["properties.City"].drop_duplicates() 
display(df_cityName)

0          м.Мукачево,
1              м.Тячів
2          м. Мукачево
3            м.Ужгород
8           с. Розівка
             ...      
1183       с. Петрушів
1184           с.Есень
1189        с. Павлово
1196    с. Чорноголова
1199       с. Кічерний
Name: properties.City, Length: 407, dtype: object